In [1]:
import pandas as pd
import numpy as np

df = pd.read_json('./../data/polish_translated/emotions/equalized_emotions_dataset_pl_with_tokens.json')
df

,emotions,text_pl,tokens,joined_tokens
0,anger,Czuję się całkowicie chciwy,"[czuję, całkowicie, chciwy]",czuję całkowicie chciwy
1,fear,Czuję się tak przytłoczony emocjami,"[czuję, przytłocić, emocja]",czuję przytłocić emocja
2,surprise,Nie czułem efektu siły g Jestem oszołomiony Po...,"[nie, czuć być, efekt, siła, g, oszołomiona, p...",nie czuć być efekt siła g oszołomiona poszedłe...
3,anger,ive just got to tell you all that I feel Racze...,"[ive, just, got, tell, you, all, that, feel, r...",ive just got tell you all that feel raczej nie...
4,anger,"Powiedziałem ci, że przepraszam, że czujesz si...","[powiedzieć być, przepraszać, czuć, sfrustrowa...",powiedzieć być przepraszać czuć sfrustrowany z...
...,...,...,...,...
39995,joy,"czuję się ostatnio dość hojny, ponieważ pracuj...","[czuć, ostatnio, hojny, pracować, znacznie, zw...",czuć ostatnio hojny pracować znacznie zwykle z...
39996,love,"Czuję, że niektóre postacie nie zasługują na w...","[czuję, niektóry, postać, nie, zasługiwać, wsp...",czuję niektóry postać nie zasługiwać wsparcie ...
39997,fear,"Czuję strach w moich kościach, którego nie moż...","[czuję, strach, kość, nie, móc, wytrząsnąć, wy...",czuję strach kość nie móc wytrząsnąć wymuszć r...
39998,joy,Czułem się bardziej kreatywny i mniej jak czys...,"[czuć być, kreatywny, czysty, redaktor]",czuć być kreatywny czysty redaktor


In [2]:
ordered_class_labels = {
    "joy": 0,
    "love": 1,
    "surprise": 2,
    "sadness": 3,
    "anger": 4,
    "fear": 5,
}

df = df.replace({"emotions": ordered_class_labels})
df

,emotions,text_pl,tokens,joined_tokens
0,4,Czuję się całkowicie chciwy,"[czuję, całkowicie, chciwy]",czuję całkowicie chciwy
1,5,Czuję się tak przytłoczony emocjami,"[czuję, przytłocić, emocja]",czuję przytłocić emocja
2,2,Nie czułem efektu siły g Jestem oszołomiony Po...,"[nie, czuć być, efekt, siła, g, oszołomiona, p...",nie czuć być efekt siła g oszołomiona poszedłe...
3,4,ive just got to tell you all that I feel Racze...,"[ive, just, got, tell, you, all, that, feel, r...",ive just got tell you all that feel raczej nie...
4,4,"Powiedziałem ci, że przepraszam, że czujesz si...","[powiedzieć być, przepraszać, czuć, sfrustrowa...",powiedzieć być przepraszać czuć sfrustrowany z...
...,...,...,...,...
39995,0,"czuję się ostatnio dość hojny, ponieważ pracuj...","[czuć, ostatnio, hojny, pracować, znacznie, zw...",czuć ostatnio hojny pracować znacznie zwykle z...
39996,1,"Czuję, że niektóre postacie nie zasługują na w...","[czuję, niektóry, postać, nie, zasługiwać, wsp...",czuję niektóry postać nie zasługiwać wsparcie ...
39997,5,"Czuję strach w moich kościach, którego nie moż...","[czuję, strach, kość, nie, móc, wytrząsnąć, wy...",czuję strach kość nie móc wytrząsnąć wymuszć r...
39998,0,Czułem się bardziej kreatywny i mniej jak czys...,"[czuć być, kreatywny, czysty, redaktor]",czuć być kreatywny czysty redaktor


In [3]:
X_train = df.loc[:29999, 'joined_tokens'].values
y_train = df.loc[:29999, 'emotions'].values
X_test = df.loc[30000:, 'joined_tokens'].values
y_test = df.loc[30000:, 'emotions'].values

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
total_reviews = df['joined_tokens'].values

tokenizer.fit_on_texts(total_reviews)

# pad sequences
max_length = max([len(s.split()) for s in total_reviews])

# define vocab size
vocab_size = len(tokenizer.word_index) + 1

X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout

EMBEDDING_DIM = 100

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
history = model.fit(X_train_pad, y_train, batch_size=128, epochs=2, validation_data=(X_test_pad, y_test))

Epoch 1/2
235/235 [==============================] - 3s 4ms/step - loss: 1.5856 - accuracy: 0.3396 - val_loss: 1.0919 - val_accuracy: 0.6671
Epoch 2/2
235/235 [==============================] - 1s 3ms/step - loss: 0.7407 - accuracy: 0.7551 - val_loss: 0.6868 - val_accuracy: 0.7778


In [26]:
assignments_df = pd.read_json("./../data/polish_translated/emotions/assignments_dataset.json")
assignments_df

,comment,emotion
0,Jak palion zrobi piosenkę to wiadomo co będzie...,joy
1,"Wiem, że palion może tego nigdy nie zobaczyć, ...",joy
2,Był kiedyś taki serial : Kasia i Tomek i wedłu...,joy
3,Mamy właśnie przykład jak powinna wyglądać pra...,joy
4,Siemka WK teraz moja kolej na nagrodę 😁 pozdra...,joy
...,...,...
1327,Uwielbiam twoje firmy zabieram się za oglądani...,joy
1328,"Dla mnie ""Film o pszczołach"" był genialny. Zar...",sadness
1329,Zawsze o 14.00 odrazu każda sobota jest lepsza...,joy
1330,Zawsze oglądane do końca! La Budda <3,love


In [27]:
assignments_df = assignments_df.replace({"emotion": ordered_class_labels})
assignments_df

,comment,emotion
0,Jak palion zrobi piosenkę to wiadomo co będzie...,0
1,"Wiem, że palion może tego nigdy nie zobaczyć, ...",0
2,Był kiedyś taki serial : Kasia i Tomek i wedłu...,0
3,Mamy właśnie przykład jak powinna wyglądać pra...,0
4,Siemka WK teraz moja kolej na nagrodę 😁 pozdra...,0
...,...,...
1327,Uwielbiam twoje firmy zabieram się za oglądani...,0
1328,"Dla mnie ""Film o pszczołach"" był genialny. Zar...",3
1329,Zawsze o 14.00 odrazu każda sobota jest lepsza...,0
1330,Zawsze oglądane do końca! La Budda <3,1


In [28]:
assignments_X_test = assignments_df['comment'].values
assignments_X_test_tokens = tokenizer.texts_to_sequences(assignments_X_test)
assignments_X_test_pad = pad_sequences(assignments_X_test_tokens, maxlen=max_length, padding='post')

assignments_y_test = assignments_df['emotion'].values

In [29]:
model.evaluate(assignments_X_test_pad, assignments_y_test)

42/42 [==============================] - 0s 4ms/step - loss: 2.2658 - accuracy: 0.1884


[2.265772581100464, 0.18843844532966614]